<a href="https://colab.research.google.com/github/Vaishnavichandur/project-by-me/blob/main/stockpriceprediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Fetch stock data
def get_stock_data(ticker, start, end):
    stock_data = yf.download(ticker, start=start, end=end)
    return stock_data[['Close']]

# Prepare dataset for LSTM
def prepare_data(data, time_step=60):
    scaler = MinMaxScaler(feature_range=(0,1))
    data_scaled = scaler.fit_transform(data)
    X, y = [], []
    for i in range(time_step, len(data_scaled)):
        X.append(data_scaled[i-time_step:i, 0])
        y.append(data_scaled[i, 0])
    return np.array(X), np.array(y), scaler

# Build LSTM model
def build_lstm_model(input_shape):
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(50, return_sequences=False),
        Dropout(0.2),
        Dense(25),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Main execution
if __name__ == "__main__":
    ticker = 'AAPL'  # Change to desired stock ticker
    start_date = '2020-01-01'
    end_date = '2024-01-01'

    data = get_stock_data(ticker, start_date, end_date)

    # Prepare training data
    X, y, scaler = prepare_data(data.values)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    # Train model
    model = build_lstm_model((X.shape[1], 1))
    model.fit(X, y, epochs=20, batch_size=32)

    # Predict next day's price
    last_60_days = data.values[-60:].reshape(1, -1)
    last_60_days_scaled = scaler.transform(last_60_days.T).T
    X_test = np.reshape(last_60_days_scaled, (1, last_60_days_scaled.shape[1], 1))
    predicted_price = model.predict(X_test)
    predicted_price = scaler.inverse_transform(predicted_price)

    print(f"Predicted closing price for next day: {predicted_price[0][0]}")

    # Plot actual vs predicted prices
    plt.plot(data.index[-100:], data.values[-100:], label='Actual Prices')
    plt.axvline(data.index[-1], color='r', linestyle='--', label='Prediction Point')
    plt.scatter(data.index[-1], predicted_price, color='g', label='Predicted Price')
    plt.legend()
    plt.show()
